# 20 · Model Baseline
Simple classification of home win vs loss.

In [ ]:
import os, glob
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report

# Load data
files = sorted(glob.glob("data/exports/**/*.parquet", recursive=True))
if not files:
    print("No parquet found. Run 00_colab_bootstrap first.")
    df = pd.DataFrame()
else:
    df = pd.concat((pd.read_parquet(p) for p in files), ignore_index=True)

assert not df.empty, "No data."

# Target: home win
df = df.dropna(subset=["home_score","away_score","home_team","away_team"])
df["home_win"] = (df["home_score"] > df["away_score"]).astype(int)

# Simple features
feat_cat = ["home_team","away_team","venue"]
feat_num = []
X = df[feat_cat + feat_num]
y = df["home_win"]

ct = ColumnTransformer([("cat", OneHotEncoder(handle_unknown="ignore"), feat_cat)], remainder="drop")
clf = Pipeline([("prep", ct), ("logreg", LogisticRegression(max_iter=1000))])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
clf.fit(X_train, y_train)
proba = clf.predict_proba(X_test)[:,1]
pred  = (proba >= 0.5).astype(int)

print("AUC:", roc_auc_score(y_test, proba))
print("ACC:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred)[:800])


In [ ]:
# Export HTML report
!jupyter nbconvert --to html --output-dir=reports --no-input notebooks/20_model_baseline.ipynb
